In [ ]:
import pandas as pd 
from pandas.tseries.offsets import BDay
from datetime import datetime, timedelta
from dateutil import relativedelta
import calendar 
import win32com.client as win32
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.base import MIMEBase
from email import encoders
import smtplib, ssl
import os
import xlsxwriter

import sys
sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases

In [ ]:
# Se True: Manda para todos os emails que estiverem na tabela 'times'.
# Se False: Manda APENAS para os nomes que estiverem na lista 'ALVOS_ESPECIFICOS'.
ENVIAR_PARA_TODOS = True

# Coloque aqui os nomes dos assessores
DESTINATARIO = ['FERNANDO DOMINGUES DA SILVA',
#'PAULO ROBERTO FARIA SILVA',
#'SAADALLAH JOSE ASSAD',
#'RODRIGO DE MELLO D’ELIA',
#'ROSANA APARECIDA PAVANI DA SILVA',
#'GABRIEL GUERRERO TORRES FONSECA',
#'CAIC ZEM GOMES',
#'RENAN BENTO DA SILVA',
#'RAFAEL PASOLD MEDEIROS',
#'FELIPE AUGUSTO CARDOSO',
#'MARCOS SOARES PEREIRA FILHO',
#'IZADORA VILLELA FREITAS',
#'GUILHERME DE LUCCA BERTELONI',
#'VITOR OLIVEIRA DOS REIS',
]

# Se True, ele apenas printa que enviaria, mas não envia
MODO_TESTE = False

In [ ]:
## Adicionar coluna de time e de pf ou pj

In [ ]:
conexao = Connect.connect_techdb()
basebtg = Connect.import_table(conexao, "base_btg")
times = Connect.import_table(conexao, "times_nova_empresa")
saldo = Connect.import_table(conexao, "saldo_conta_corrente")
posicoes = Connect.import_table(conexao, "posicao")
conexao.close()

In [ ]:
basebtg = basebtg.merge(times, on='Assessor', how='left')

In [ ]:
## Clientes com Conta, Assessor e Nome completo
basebtg = basebtg[['Conta', 'Assessor', 'Tipo', 'TIME', 'Aniversário']]
nomes_clientes = pd.read_excel(r"C:\Scripts\nomes_clientes\Nomes_clientes2.xlsx")
nomes_clientes.rename(columns={"Código":"Conta"}, inplace=True)
basebtg = basebtg.merge(nomes_clientes, on='Conta', how='left')

In [ ]:
## Criar novo dataframe com os clientes + saldo em cc
saldo.rename(columns={"CONTA":"Conta"}, inplace=True)
table = pd.merge(basebtg, saldo[['Conta', 'SALDO']], on='Conta', how='left')

In [ ]:
for coluna in posicoes.columns:
    posicoes.rename(columns={coluna:coluna.upper()}, inplace=True)

In [ ]:
## Adiconar CDB Plus
cdb_plus_btg = posicoes[posicoes['PRODUTO'] == 'BTG CDB Plus FIRF CrPr']
cdb_plus_btg = cdb_plus_btg.iloc[:,[0, 16]].groupby("CONTA").sum().reset_index()
cdb_plus_btg.rename(columns={"VALOR LÍQUIDO":'CDB Plus'}, inplace=True)

table.rename(columns={"Conta":"CONTA"}, inplace=True)
table = table.merge(cdb_plus_btg[['CONTA', 'CDB Plus']], on='CONTA', how='left')

In [ ]:
## Adiconar LFTs
lft = posicoes[posicoes['PRODUTO'] == 'LFT']
lft = lft.iloc[:,[0, 16]].groupby("CONTA").sum().reset_index()
lft.rename(columns={"VALOR LÍQUIDO":'LFT'}, inplace=True)

table.rename(columns={"Conta":"CONTA"}, inplace=True)
table = table.merge(lft[['CONTA', 'LFT']], on='CONTA', how='left')

In [ ]:
## Adicionar CDBs de líquidez diária
cdb = posicoes[posicoes['PRODUTO'] == "CDB"]

conexao = Connect.connect_techdb()
rf_coe = Connect.import_table(conexao, "renda_fixa")
conexao.close()

for coluna in rf_coe.columns:
    rf_coe.rename(columns={coluna:coluna.upper()}, inplace=True)

rf_coe = rf_coe[['ATIVO', 'LIQUIDEZ']]

cdb = cdb.merge(rf_coe, on='ATIVO', how='left')
cdb_liq_diaria = cdb[cdb['LIQUIDEZ'] == "Liquidez Diaria"]
cdb_liq_diaria = cdb_liq_diaria[['CONTA', 'VALOR LÍQUIDO']].groupby("CONTA").sum().reset_index()
cdb_liq_diaria.rename(columns={"VALOR LÍQUIDO":"CDB Líquidez Diária"}, inplace=True)
table = table.merge(cdb_liq_diaria, on='CONTA', how='left')

In [ ]:
hoje = datetime.today()
res = calendar.monthrange(hoje.year, hoje.month)
day = res[1]
ultimo_dia_do_mes = datetime(hoje.year, hoje.month, day).strftime("%Y-%m-%d")

In [ ]:
## Adicionar os vencimentos até o fim do mes
vencimento = posicoes[posicoes['VENCIMENTO'] <= ultimo_dia_do_mes]
vencimento = vencimento[vencimento['MERCADO'] != "Valor em Trânsito"]
vencimento = vencimento[['CONTA', 'VALOR LÍQUIDO']].groupby("CONTA").sum().reset_index()
vencimento.rename(columns={"VALOR LÍQUIDO":"Vencimentos Mes Atual"}, inplace=True)
table = table.merge(vencimento, on='CONTA', how='left')

In [ ]:
## Adicionar fundos d+0 e d+1
#fundos = fundos_posicao.copy()
#fundos.rename(columns={"'DL_D_ContaAssessor'[NR_CONTA]":"CONTA"}, inplace=True)
#for coluna in fundos.columns:
#    fundos.rename(columns={coluna:coluna.upper()}, inplace=True)
#fundos.rename(columns={"RESGATE (D+)":"REGATE (D+)"}, inplace=True)
#fundos = fundos[(fundos['REGATE (D+)'] == 0) | (fundos['REGATE (D+)'] == 1)]
#fundos_liq = fundos[['CONTA', 'VALOR LÍQUIDO']].groupby("CONTA").sum().reset_index()
#fundos_liq.rename(columns={"VALOR LÍQUIDO":"Fundos d+0/d+1"}, inplace=True)
#fundos_liq['CONTA'] = fundos_liq['CONTA'].astype(str)
#table = table.merge(fundos_liq, on='CONTA', how='left')

In [ ]:
table.fillna(0, inplace=True)

In [ ]:
conexao = Connect.connect_techdb()
base_pl = Connect.import_table(conexao, "base_btg") 
conexao.close()

In [ ]:
# Mudamos todas as referências abaixo para base_pl
base_pl = base_pl[['Conta', 'PL Total', 'PL Declarado']]
base_pl.rename(columns={"Conta":"CONTA"}, inplace=True)
base_pl['% share of wallet'] = base_pl['PL Total']/base_pl['PL Declarado']
table = table.merge(base_pl[['CONTA', "% share of wallet"]], on='CONTA', how='left')

In [ ]:
conexao = Connect.connect_techdb()
proventos = Connect.import_table(conexao, 'proventos_futuros_rf')
conexao.close()

In [ ]:
proventos.head(2)

In [ ]:
proventos.rename(columns={"Mes":"Data", "Conta":"CONTA"}, inplace=True)
proventos_por_mes = proventos.iloc[:,[0, 7, 9]].groupby(["CONTA", "Data"]).sum().reset_index()

In [ ]:
proventos_no_mes = \
    proventos_por_mes[
        (proventos_por_mes['Data'] == datetime.today().strftime("%Y-%m"))
    ]

In [ ]:
proventos_no_mes.rename(columns={"Total proventos":"Proventos no Mês"}, inplace=True)

In [ ]:
table = table.merge(proventos_no_mes[['CONTA', 'Proventos no Mês']], on='CONTA', how='left')

In [ ]:
table['Proventos no Mês'].fillna(0, inplace=True)
table['Soma Liquidez'] = table['SALDO'] + table['LFT'] + table['CDB Líquidez Diária'] + table['Vencimentos Mes Atual'] + table['Proventos no Mês']

In [ ]:
pl_clientes = base_pl.loc[:,["CONTA", "PL Total"]]

In [ ]:
table = table.merge(pl_clientes)
table["Líquidez da Carteira"] = table["Soma Liquidez"]/table["PL Total"]

In [ ]:
liquidez_assessor = table.loc[:,["Assessor", "Soma Liquidez", "PL Total"]].groupby("Assessor").sum().reset_index()
liquidez_assessor["Liquidez da carteira"] = liquidez_assessor["Soma Liquidez"]/liquidez_assessor["PL Total"]
liquidez_assessor = liquidez_assessor.sort_values("Liquidez da carteira", ascending=False)
liquidez_assessor[(liquidez_assessor['Assessor'].isin(times['Assessor'])) & (liquidez_assessor['Assessor'] != "DIGITAL")].loc[:,["Assessor", "Liquidez da carteira"]]

In [ ]:
liquidez_assessor[(liquidez_assessor['Assessor'].isin(times['Assessor'])) & (liquidez_assessor['Assessor'] != "DIGITAL")]['Liquidez da carteira'].mean()

In [ ]:
conexao = Connect.connect_techdb()
table.to_sql(con=conexao, name="liquidez_clientes", 
             if_exists='replace', 
             index=False,
             schema='principal')
conexao.close()
table.to_excel(r"C:\Scripts\liquidez\pacote_liquidez.xlsx", header=True, index=False)

In [ ]:
infos = table[[ 
    'CONTA', 'Assessor', 'Nome', 'SALDO','PL Total', 'CDB Líquidez Diária', 'LFT', 'CDB Plus',
    'Vencimentos Mes Atual', 'Proventos no Mês', 'Soma Liquidez',
    'Líquidez da Carteira'
]]

infos.to_excel(
    r"C:\Scripts\liquidez\infos_todos.xlsx",
    index=False
)

In [ ]:
infos

In [ ]:
# 1. Definir datas alvo 
hoje_dt = datetime.today()
dia_semana = hoje_dt.weekday()

# Lista 
datas_alvo = []

In [ ]:
if dia_semana == 4: # Sexta-feira
    print("Hoje é Sexta-feira. Buscando aniversariantes de Sex, Sáb e Dom.")
    datas_alvo.append((hoje_dt.day, hoje_dt.month)) # Sexta
    sabado = hoje_dt + timedelta(days=1)
    datas_alvo.append((sabado.day, sabado.month))   # Sabado
    domingo = hoje_dt + timedelta(days=2)
    datas_alvo.append((domingo.day, domingo.month)) # Domingo
else:
    print("Dia de semana comum. Buscando aniversariantes de hoje.")
    datas_alvo.append((hoje_dt.day, hoje_dt.month))

In [ ]:
print(basebtg.columns)

In [ ]:
df_niver = basebtg[['Conta', 'Assessor', 'Tipo', 'Nome', 'Aniversário']].copy()

# Converter para datetime e remover erros (NaT)
df_niver['Aniversário'] = pd.to_datetime(df_niver['Aniversário'], errors='coerce')
df_niver = df_niver.dropna(subset=['Aniversário'])

# Filtrar apenas PF
df_niver = df_niver[df_niver['Tipo'] == 'PF']

# Criar colunas auxiliares de Dia e Mês para comparação
df_niver['Dia'] = df_niver['Aniversário'].dt.day
df_niver['Mes'] = df_niver['Aniversário'].dt.month

In [ ]:
caminho_base = r"C:\Scripts\liquidez"
outlook = win32.Dispatch('outlook.application')

if not os.path.exists(caminho_base):
    os.makedirs(caminho_base)

if ENVIAR_PARA_TODOS:
    lista_assessores = times['Assessor'].unique()
else:
    lista_assessores = DESTINATARIO

print(f"Iniciando processamento para {len(lista_assessores)} assessores...\n")

for nome_assessor in lista_assessores:
    dados_assessor = times[times['Assessor'] == nome_assessor]
    if dados_assessor.empty:
        continue 
    
    if nome_assessor == 'FERNANDO DOMINGUES DA SILVA':
        email_destino = 'murilo.gino@atriacm.com.br'
    else:
        email_destino = dados_assessor['Email'].iloc[0]
    
    # Filtrar dados de liquidez
    infos_filtrada = infos[infos['Assessor'] == nome_assessor].copy()
    
    if infos_filtrada.empty:
        print(f"[AVISO] {nome_assessor} sem dados de liquidez. Pulando.")
        continue

    # --- PROCESSAMENTO EXCEL (MANTIDO IGUAL) ---
    infos_filtrada = infos_filtrada.drop(columns=['Assessor'])
    if 'SALDO' in infos_filtrada.columns:
        infos_filtrada = infos_filtrada.sort_values(by='SALDO', ascending=False)
    
    if nome_assessor == 'CAIC ZEM GOMES':
        infos_filtrada = infos_filtrada.iloc[:, :4]

    nome_arquivo = f"infos_{nome_assessor}.xlsx"
    caminho_completo = os.path.join(caminho_base, nome_arquivo)
    
    with pd.ExcelWriter(caminho_completo, engine='xlsxwriter') as writer:
        infos_filtrada.to_excel(writer, index=False, sheet_name='Relatorio')
        workbook = writer.book
        worksheet = writer.sheets['Relatorio']
        max_row = len(infos_filtrada)
        max_col = infos_filtrada.shape[1] - 1
        worksheet.autofilter(0, 0, max_row, max_col)
        fmt_moeda = workbook.add_format({'num_format': 'R$ #,##0.00'})   
        fmt_porcentagem = workbook.add_format({'num_format': '0.00%'})
        worksheet.set_column('A:Z', 23) 
        worksheet.set_column('B:B', 40) 
        worksheet.set_column('A:A', 14) 
        worksheet.set_column('C:J', 23, fmt_moeda) 
        worksheet.set_column('K:K', 23, fmt_porcentagem)

    # Filtra clientes deste assessor
    niver_assessor = df_niver[df_niver['Assessor'] == nome_assessor]
    
    lista_html_niver = ""
    tem_aniversariante = False

    if not niver_assessor.empty:
        # Filtra quem faz aniversário nas datas alvo (Dia e Mes batem)
        # Usamos uma query ou list comprehension
        
        # Cria uma coluna temporária "Check" para verificar se a tupla (Dia, Mes) está na lista alvo
        # Maneira eficiente em pandas:
        filtro_data = niver_assessor.apply(lambda x: (x['Dia'], x['Mes']) in datas_alvo, axis=1)
        aniversariantes_hoje = niver_assessor[filtro_data].copy()

        if not aniversariantes_hoje.empty:
            tem_aniversariante = True
            # Ordenar por dia
            aniversariantes_hoje.sort_values(by=['Mes', 'Dia'], inplace=True)
            
            # --- Cabeçalho da Tabela ---
            lista_html_niver += """
            <div style="margin-top: 20px; padding: 15px; background-color: #f9f9f9; border-left: 5px solid #2C3E50; font-family: Arial, sans-serif;">
            <h3 style="color: #2C3E50; margin-top: 0;">Aniversariantes</h3>
            <table style="border-collapse: collapse; width: 100%;">
                <tr style="background-color: #e0e0e0; font-size: 12px;">
                    <th style="padding: 5px; text-align: left;">Data</th>
                    <th style="padding: 5px; text-align: left;">Conta</th>
                    <th style="padding: 5px; text-align: left;">Cliente</th>
                </tr>
            """
            
            # --- Linhas da Tabela ---
            for idx, row in aniversariantes_hoje.iterrows():
                # Formata a data (dd/mm)
                dia_nasc = f"{row['Dia']:02d}/{row['Mes']:02d}"
                nome_cli = row['Nome']
                conta_cli = row['Conta'] # Pega o número da conta
                
                # Label de dia da semana (Hoje/Sáb/Dom)
                label_dia = ""
                if dia_semana == 4: # Se é sexta-feira
                    if row['Dia'] == hoje_dt.day: label_dia = " (Hoje)"
                    elif row['Dia'] == (hoje_dt + timedelta(days=1)).day: label_dia = " (Sáb)"
                    elif row['Dia'] == (hoje_dt + timedelta(days=2)).day: label_dia = " (Dom)"

                # Adiciona a linha com 3 colunas: Data | Conta | Nome
                lista_html_niver += f"""
                <tr style="border-bottom: 1px solid #ddd;">
                    <td style="padding: 8px; color: #555; white-space: nowrap;"><b>{dia_nasc}</b><span style="font-size:10px; color:#777">{label_dia}</span></td>
                    <td style="padding: 8px; color: #333;">{conta_cli}</td>
                    <td style="padding: 8px; color: #333;">{nome_cli}</td>
                </tr>
                """
            lista_html_niver += "</table></div>"
        
    # Se não tiver aniversariante, a variável lista_html_niver fica vazia e não polui o email
    
    
    if MODO_TESTE:
        print(f"--- [TESTE] Arquivo: {nome_arquivo} | Para: {email_destino}")
        if tem_aniversariante:
            print("    [TESTE] Inclui aniversariantes no corpo.")
    else:
        try:
            email = outlook.CreateItem(0)
            email.To = email_destino
            email.Subject = "Relatório de Saldo, Liquidez e Aniversariantes"
            
            # Inserimos a variável {lista_html_niver} no final do HTMLBody
            email.HTMLBody = f"""
            <div style="font-family: Arial, sans-serif; color: #333;">
                <p>Olá, {nome_assessor.split()[0].title()}</p>
                <p>Segue em anexo a planilha formatada com os saldos das contas atualizados.</p>
                
                {lista_html_niver}
                
                <br>
                <p>Qualquer dúvida ou problema estou a disposição!</p>
                <p>Abs,</p>
            </div>
            """
            email.Attachments.Add(caminho_completo)
            email.Send()
            print(f"[SUCESSO] Enviado para: {nome_assessor}")
        except Exception as e:
            print(f"[FALHA] Erro ao enviar para {nome_assessor}: {e}")

print("Fim do processamento.")